# Section A: Problem Statement – Enhancing Neural Network Performance with Particle Swarm Optimization

In [11]:
# so,I have Chosen digits dataset from scikit learn library

In [10]:
# Step 1: Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import random
import warnings
import gc

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore')


In [13]:
import pandas as pd
from sklearn.datasets import load_digits

# Load the digits dataset (MNIST equivalent for digits classification)
digits = load_digits()

# Convert to a DataFrame for better visualization
df_digits = pd.DataFrame(digits.data, columns=[f'feature_{i+1}' for i in range(digits.data.shape[1])])
df_digits['target'] = digits.target

# Display the first few rows of the dataset
df_digits.head()


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [4]:
# Step 3: Define a model building function
def build_model(learning_rate=0.001, num_neurons=64, solver='adam'):
    model = MLPClassifier(hidden_layer_sizes=(int(num_neurons),),
                          learning_rate_init=learning_rate,
                          solver=solver,
                          max_iter=200,
                          random_state=42)
    return model


In [5]:
# Step 4: Random Search (Traditional Optimization)
learning_rates = [0.01, 0.001, 0.0001]
neurons = [32, 64, 128]
solvers = ['adam', 'sgd']

best_acc_random = 0
best_params_random = {}

for lr in learning_rates:
    for neuron in neurons:
        for solver in solvers:
            model = build_model(learning_rate=lr, num_neurons=neuron, solver=solver)
            model.fit(x_train, y_train)
            val_preds = model.predict(x_val)
            val_acc = accuracy_score(y_val, val_preds)

            if val_acc > best_acc_random:
                best_acc_random = val_acc
                best_params_random = {'learning_rate': lr, 'neurons': neuron, 'solver': solver}

            del model
            gc.collect()

print("\nBest Parameters (Random Search):", best_params_random)
print("Best Validation Accuracy (Random Search): {:.4f}".format(best_acc_random))



Best Parameters (Random Search): {'learning_rate': 0.01, 'neurons': 32, 'solver': 'adam'}
Best Validation Accuracy (Random Search): 0.9826


In [14]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score

# Define hyperparameter grid
param_grid = {
    'learning_rate_init': [0.001, 0.01, 0.1],  # Different learning rates to try
    'hidden_layer_sizes': [(50,), (100,), (150,)]  # Different number of neurons
}

# Iterate over all parameter combinations
best_acc = 0
best_params = None

for params in ParameterGrid(param_grid):
    # Build and train the model
    model = MLPClassifier(hidden_layer_sizes=params['hidden_layer_sizes'],
                          learning_rate_init=params['learning_rate_init'],
                          max_iter=200,
                          random_state=42)
    model.fit(x_train, y_train)  # Train the model
    
    # Evaluate the model
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    
    # Update best model if current one is better
    if acc > best_acc:
        best_acc = acc
        best_params = params

# Print the best hyperparameters and accuracy
print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_acc:.4f}")


Best Hyperparameters: {'hidden_layer_sizes': (150,), 'learning_rate_init': 0.001}
Best Accuracy: 0.9778


In [6]:
# Step 5: PSO Optimization

# Simple PSO (no pyswarm needed!)
def objective_function(params):
    learning_rate, num_neurons = params
    learning_rate = float(learning_rate)
    num_neurons = int(num_neurons)
    if num_neurons < 10:
        num_neurons = 10

    model = build_model(learning_rate=learning_rate, num_neurons=num_neurons)
    model.fit(x_train, y_train)
    val_preds = model.predict(x_val)
    val_acc = accuracy_score(y_val, val_preds)

    del model
    gc.collect()

    return -val_acc  # PSO minimizes

# Basic PSO algorithm
def simple_pso(objective_function, lb, ub, swarmsize=5, maxiter=3):
    dim = len(lb)
    X = np.random.uniform(low=lb, high=ub, size=(swarmsize, dim))
    V = np.zeros((swarmsize, dim))
    pbest = X.copy()
    pbest_val = np.array([objective_function(x) for x in X])
    gbest = pbest[np.argmin(pbest_val)]
    gbest_val = np.min(pbest_val)

    w = 0.5
    c1 = 1
    c2 = 2

    for it in range(maxiter):
        for i in range(swarmsize):
            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            V[i] = (w * V[i]) + (c1 * r1 * (pbest[i] - X[i])) + (c2 * r2 * (gbest - X[i]))
            X[i] = X[i] + V[i]
            X[i] = np.clip(X[i], lb, ub)
            val = objective_function(X[i])

            if val < pbest_val[i]:
                pbest[i] = X[i]
                pbest_val[i] = val

        gbest = pbest[np.argmin(pbest_val)]
        gbest_val = np.min(pbest_val)
        
    return gbest, gbest_val

# Define the search space
lb = [0.0001, 32]
ub = [0.01, 128]

best_params_pso, best_score_pso = simple_pso(objective_function, lb, ub, swarmsize=5, maxiter=3)

# Extract best parameters
best_learning_rate_pso = best_params_pso[0]
best_neurons_pso = int(best_params_pso[1])

print("\nBest Parameters (PSO):")
print(f"Learning Rate: {best_learning_rate_pso:.5f}")
print(f"Neurons: {best_neurons_pso}")
print(f"Best Validation Accuracy (PSO): {-best_score_pso:.4f}")



Best Parameters (PSO):
Learning Rate: 0.00589
Neurons: 128
Best Validation Accuracy (PSO): 0.9896


In [7]:
# Step 6: Final Model Training and Evaluation

# Train final model from Random Search
final_model_random = build_model(
    learning_rate=best_params_random['learning_rate'],
    num_neurons=best_params_random['neurons'],
    solver=best_params_random['solver']
)
final_model_random.fit(np.vstack((x_train, x_val)), np.hstack((y_train, y_val)))
test_preds_random = final_model_random.predict(x_test)
test_acc_random = accuracy_score(y_test, test_preds_random)

# Train final model from PSO
final_model_pso = build_model(
    learning_rate=best_learning_rate_pso,
    num_neurons=best_neurons_pso
)
final_model_pso.fit(np.vstack((x_train, x_val)), np.hstack((y_train, y_val)))
test_preds_pso = final_model_pso.predict(x_test)
test_acc_pso = accuracy_score(y_test, test_preds_pso)

print("\nTest Accuracy (Random Search): {:.4f}".format(test_acc_random))
print("Test Accuracy (PSO): {:.4f}".format(test_acc_pso))


Test Accuracy (Random Search): 0.9722
Test Accuracy (PSO): 0.9778


In [8]:
if test_acc_pso > test_acc_random:
    print("\nConclusion: The PSO-optimized model outperformed the Random Search model.")
else:
    print("\nConclusion: The Random Search model performed better, but PSO achieved competitive results.")



Conclusion: The PSO-optimized model outperformed the Random Search model.


In [9]:
from statsmodels.stats.contingency_tables import mcnemar

# Generate confusion matrix for McNemar's test
contingency_table = np.zeros((2, 2))

# Compare predictions of Random Search model and PSO model
for i in range(len(y_test)):
    random_search_correct = (test_preds_random[i] == y_test[i])
    pso_correct = (test_preds_pso[i] == y_test[i])
    
    if random_search_correct and pso_correct:
        contingency_table[0, 0] += 1  # Both correct
    elif random_search_correct and not pso_correct:
        contingency_table[0, 1] += 1  # Random Search correct, PSO wrong
    elif not random_search_correct and pso_correct:
        contingency_table[1, 0] += 1  # Random Search wrong, PSO correct
    else:
        contingency_table[1, 1] += 1  # Both wrong

# Apply McNemar's test
result = mcnemar(contingency_table, exact=True)

# Print results
print("\nComparison of Random Search vs PSO-Optimized Model:")
print(f"Random Search Test Accuracy: {test_acc_random:.4f}")
print(f"PSO-Optimized Test Accuracy: {test_acc_pso:.4f}")

# McNemar's Test results
print("\nMcNemar's Test Result:")
print(f"p-value: {result.pvalue:.5f}")
if result.pvalue < 0.05:
    print("The improvement from PSO optimization is **statistically significant**.")
else:
    print("No **statistically significant** improvement from PSO optimization.")



Comparison of Random Search vs PSO-Optimized Model:
Random Search Test Accuracy: 0.9722
PSO-Optimized Test Accuracy: 0.9778

McNemar's Test Result:
p-value: 0.68750
No **statistically significant** improvement from PSO optimization.
